# Example Predictor: Linear Rollout Predictor

This example contains basic functionality for training and evaluating a linear predictor that rolls out predictions day-by-day.

First, a training data set is created from historical case and npi data.

Second, a linear model is trained to predict future cases from prior case data along with prior and future npi data.
The model is an off-the-shelf sklearn Lasso model, that uses a positive weight constraint to enforce the assumption that increased npis has a negative correlation with future cases.

Third, a sample evaluation set is created, and the predictor is applied to this evaluation set to produce prediction results in the correct format.

## Training

In [15]:
import pickle
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

### Copy the data locally

In [16]:
# Main source for the training data
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
# Local file
DATA_FILE = 'data/OxCGRT_latest.csv'

In [17]:
import os
import urllib.request
if not os.path.exists('data'):
    os.mkdir('data')
urllib.request.urlretrieve(DATA_URL, DATA_FILE)

('data/OxCGRT_latest.csv', <http.client.HTTPMessage at 0x7fac08dffaf0>)

In [18]:
# Load historical data from local file
df = pd.read_csv(DATA_FILE, 
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)

In [19]:
df.columns

Index(['CountryName', 'CountryCode', 'RegionName', 'RegionCode',
       'Jurisdiction', 'Date', 'C1_School closing', 'C1_Flag',
       'C2_Workplace closing', 'C2_Flag', 'C3_Cancel public events', 'C3_Flag',
       'C4_Restrictions on gatherings', 'C4_Flag', 'C5_Close public transport',
       'C5_Flag', 'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'E1_Income support', 'E1_Flag',
       'E2_Debt/contract relief', 'E3_Fiscal measures',
       'E4_International support', 'H1_Public information campaigns',
       'H1_Flag', 'H2_Testing policy', 'H3_Contact tracing',
       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
       'H6_Facial Coverings', 'H6_Flag', 'M1_Wildcard', 'ConfirmedCases',
       'ConfirmedDeaths', 'StringencyIndex', 'StringencyIndexForDisplay',
       'StringencyLegacyIndex', 'StringencyLegacyIndexForDisplay',
       'GovernmentResponseIndex', 'Gove

In [20]:
# For testing, restrict training data to that before a hypothetical predictor submission date
HYPOTHETICAL_SUBMISSION_DATE = np.datetime64("2020-07-31")
df = df[df.Date <= HYPOTHETICAL_SUBMISSION_DATE]

In [21]:
# Add RegionID column that combines CountryName and RegionName for easier manipulation of data
df['GeoID'] = df['CountryName'] + '__' + df['RegionName'].astype(str)

In [22]:
# Add new cases column
df['NewCases'] = df.groupby('GeoID').ConfirmedCases.diff().fillna(0)

In [23]:
# Keep only columns of interest
id_cols = ['CountryName',
           'RegionName',
           'GeoID',
           'Date']
cases_col = ['NewCases']
npi_cols = ['C1_School closing',
            'C2_Workplace closing',
            'C3_Cancel public events',
            'C4_Restrictions on gatherings',
            'C5_Close public transport',
            'C6_Stay at home requirements',
            'C7_Restrictions on internal movement',
            'C8_International travel controls',
            'H1_Public information campaigns',
            'H2_Testing policy',
            'H3_Contact tracing',
            'H6_Facial Coverings']
df = df[id_cols + cases_col + npi_cols]

In [24]:
# Fill any missing case values by interpolation and setting NaNs to 0
df.update(df.groupby('GeoID').NewCases.apply(
    lambda group: group.interpolate()).fillna(0))

In [25]:
# Fill any missing NPIs by assuming they are the same as previous day
for npi_col in npi_cols:
    df.update(df.groupby('GeoID')[npi_col].ffill().fillna(0))

In [26]:
list_countries = sorted(list(set(df.CountryName)))
hist_ips_df = pd.read_csv("../../../validation/data/2020-09-30_historical_ip.csv",
                              parse_dates=['Date'],
                              encoding="ISO-8859-1",
                              dtype={"RegionName": str},
                              error_bad_lines=True)
hist_ips_df = hist_ips_df[hist_ips_df.CountryName.isin(list_countries)]
hist_ips_df.to_csv("../../../validation/data/2020-09-30_historical_ip.csv" , index = False)

In [27]:
hist_ips_df

,CountryName,RegionName,Date,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings
0,Aruba,NaN,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,NaN,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,NaN,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aruba,NaN,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aruba,NaN,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64385,Zimbabwe,NaN,2020-09-26,2.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,3.0
64386,Zimbabwe,NaN,2020-09-27,2.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,3.0
64387,Zimbabwe,NaN,2020-09-28,2.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,3.0
64388,Zimbabwe,NaN,2020-09-29,2.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,1.0


In [28]:
# Set number of past days to use to make predictions
nb_lookback_days = 30

# Create training data across all countries for predicting one day ahead
X_cols = cases_col + npi_cols
y_col = cases_col
X_samples = []
y_samples = []
geo_ids = df.GeoID.unique()
for g in geo_ids:
    gdf = df[df.GeoID == g]
    all_case_data = np.array(gdf[cases_col])
    all_npi_data = np.array(gdf[npi_cols])

    # Create one sample for each day where we have enough data
    # Each sample consists of cases and npis for previous nb_lookback_days
    nb_total_days = len(gdf)
    for d in range(nb_lookback_days, nb_total_days - 1):
        X_cases = all_case_data[d-nb_lookback_days:d]

        # Take negative of npis to support positive
        # weight constraint in Lasso.
        X_npis = -all_npi_data[d - nb_lookback_days:d]

        # Flatten all input data so it fits Lasso input format.
        X_sample = np.concatenate([X_cases.flatten(),
                                   X_npis.flatten()])
        y_sample = all_case_data[d + 1]
        X_samples.append(X_sample)
        y_samples.append(y_sample)

X_samples = np.array(X_samples)
y_samples = np.array(y_samples).flatten()

In [29]:
# Helpful function to compute mae
def mae(pred, true):
    return np.mean(np.abs(pred - true))

In [30]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_samples,
                                                    y_samples,
                                                    test_size=0.2,
                                                    random_state=301)

In [31]:
# Create and train Lasso model.
# Set positive=True to enforce assumption that cases are positively correlated
# with future cases and npis are negatively correlated.
model = Lasso(alpha=0.1,
              precompute=True,
              max_iter=10000,
              positive=True,
              selection='random')
# Fit model
model.fit(X_train, y_train)

Lasso(alpha=0.1, max_iter=10000, positive=True, precompute=True,
      selection='random')

In [32]:
# Evaluate model
train_preds = model.predict(X_train)
train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
print('Train MAE:', mae(train_preds, y_train))

test_preds = model.predict(X_test)
test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
print('Test MAE:', mae(test_preds, y_test))

Train MAE: 140.7093913629205
Test MAE: 152.4940063194295


In [33]:
# Inspect the learned feature coefficients for the model
# to see what features it's paying attention to.

# Give names to the features
x_col_names = []
for d in range(-nb_lookback_days, 0):
    x_col_names.append('Day ' + str(d) + ' ' + cases_col[0])
for d in range(-nb_lookback_days, 1):
    for col_name in npi_cols:
        x_col_names.append('Day ' + str(d) + ' ' + col_name)

# View non-zero coefficients
for (col, coeff) in zip(x_col_names, list(model.coef_)):
    if coeff != 0.:
        print(col, coeff)
print('Intercept', model.intercept_)

Day -7 NewCases 0.0010599800384059794
Day -6 NewCases 0.439769568825458
Day -5 NewCases 0.2168537668667522
Day -4 NewCases 0.0593830010565273
Day -3 NewCases 0.0689558246877214
Day -2 NewCases 0.05235638861100778
Day -1 NewCases 0.23820118873827473
Day -26 C6_Stay at home requirements 4.316095256933905
Day -22 C2_Workplace closing 9.716767615360357
Day -17 C2_Workplace closing 5.764854338654872
Intercept 26.551692081205488


In [34]:
# Save model to file
if not os.path.exists('models'):
    os.mkdir('models')
with open('models/model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

## Evaluation

Now that the predictor has been trained and saved, this section contains the functionality for evaluating it on sample evaluation data.

In [35]:
# Reload the module to get the latest changes
import predict
from importlib import reload
reload(predict)
from predict import predict_df

In [36]:
%%time
preds_df = predict_df("2020-08-01", "2020-08-31", path_to_ips_file="../../../validation/data/2020-09-30_historical_ip.csv", verbose=True)


Predicting for Aruba__nan
2020-08-01: 58.82300181119936
2020-08-02: 71.31582751123675
2020-08-03: 78.99398601057783
2020-08-04: 90.2292787574965
2020-08-05: 87.81074037842762
2020-08-06: 98.75165916655027
2020-08-07: 131.03212665889947
2020-08-08: 147.01596584944787
2020-08-09: 158.95052220908332
2020-08-10: 169.93044405023159
2020-08-11: 177.51070366204908
2020-08-12: 193.47238559050163
2020-08-13: 216.81091172018114
2020-08-14: 234.03207239542056
2020-08-15: 248.55331845472057
2020-08-16: 261.9562079357195
2020-08-17: 275.2890400534381
2020-08-18: 293.2791763110071
2020-08-19: 314.063995759744
2020-08-20: 332.4192109437261
2020-08-21: 349.22265464569745
2020-08-22: 365.4886603628373
2020-08-23: 382.5217832856015
2020-08-24: 407.87717354169223
2020-08-25: 430.06812302503084
2020-08-26: 450.55999413363935
2020-08-27: 465.98324867588303
2020-08-28: 484.63064350072614
2020-08-29: 515.333850339269
2020-08-30: 541.884867792765
2020-08-31: 566.2481063041456

Predicting for Afghanistan__nan

2020-08-27: 4219.0799640306805
2020-08-28: 4269.772841948303
2020-08-29: 4316.552979599862
2020-08-30: 4406.590881825113
2020-08-31: 4545.533550777108

Predicting for Burundi__nan
2020-08-01: 56.71653989808276
2020-08-02: 66.53450387794538
2020-08-03: 74.43623702468685
2020-08-04: 80.17767626000114
2020-08-05: 84.26311210390307
2020-08-06: 96.32427742076659
2020-08-07: 127.34120519668448
2020-08-08: 142.0748925963046
2020-08-09: 153.01310316586907
2020-08-10: 162.66425470459734
2020-08-11: 172.81195279960673
2020-08-12: 189.3982314703651
2020-08-13: 212.04358763602806
2020-08-14: 228.46329012265863
2020-08-15: 242.22523125849693
2020-08-16: 255.3659341268916
2020-08-17: 269.76324989620053
2020-08-18: 282.2554980083099
2020-08-19: 301.24530516250917
2020-08-20: 318.42507983480346
2020-08-21: 334.14702778799665
2020-08-22: 349.7583512092749
2020-08-23: 355.95010236676757
2020-08-24: 369.97362647966247
2020-08-25: 387.73821214585786
2020-08-26: 405.0426177424877
2020-08-27: 413.1147539789

2020-08-21: 2086.527542413211
2020-08-22: 2103.015368161686
2020-08-23: 2084.4130534583205
2020-08-24: 2114.0616401923467
2020-08-25: 2203.930050514393
2020-08-26: 2285.7782580519006
2020-08-27: 2331.7598239931694
2020-08-28: 2358.2345491490482
2020-08-29: 2376.1950080085894
2020-08-30: 2422.3975520751515
2020-08-31: 2496.201418727858

Predicting for Belize__nan
2020-08-01: 90.22937837977064
2020-08-02: 115.06560350944511
2020-08-03: 115.61825739210916
2020-08-04: 130.070332489382
2020-08-05: 133.15471908336298
2020-08-06: 123.64258206413754
2020-08-07: 167.556225321515
2020-08-08: 189.72708264968372
2020-08-09: 200.23796305841495
2020-08-10: 213.3907250193884
2020-08-11: 220.51956949812467
2020-08-12: 230.29061826461952
2020-08-13: 258.63209663001896
2020-08-14: 279.24321308382474
2020-08-15: 294.23185787188856
2020-08-16: 308.7570812720045
2020-08-17: 321.3738958954195
2020-08-18: 343.612605706893
2020-08-19: 368.40580821667356
2020-08-20: 389.55299855378115
2020-08-21: 407.934326503

2020-08-05: 79.93587549435514
2020-08-06: 93.87888123272586
2020-08-07: 124.19630358468919
2020-08-08: 140.10959520464306
2020-08-09: 149.3367724208934
2020-08-10: 157.99380983862216
2020-08-11: 168.74565951227913
2020-08-12: 186.05291690110965
2020-08-13: 208.6816611280192
2020-08-14: 225.26474931315343
2020-08-15: 238.18354395645696
2020-08-16: 250.86564874924767
2020-08-17: 265.5408424241491
2020-08-18: 284.10579018176736
2020-08-19: 304.5038683185075
2020-08-20: 322.2179627625344
2020-08-21: 338.1061018353781
2020-08-22: 354.10039852310916
2020-08-23: 371.6679189029829
2020-08-24: 391.4406771389861
2020-08-25: 411.94822948504844
2020-08-26: 431.286699276606
2020-08-27: 445.53173096541593
2020-08-28: 463.3859168316008
2020-08-29: 482.96633528189386
2020-08-30: 503.8570677664827
2020-08-31: 525.1686641194982

Predicting for Canada__nan
2020-08-01: 3992.9377742216093
2020-08-02: 4752.323501744425
2020-08-03: 4948.523404265031
2020-08-04: 4993.319394387919
2020-08-05: 4176.647142920733

2020-08-26: 512.2041991144488
2020-08-27: 538.70342921454
2020-08-28: 562.9715627608418
2020-08-29: 592.3640497967374
2020-08-30: 622.0815525285453
2020-08-31: 653.059860479856

Predicting for Colombia__nan
2020-08-01: 6077.205006743495
2020-08-02: 7487.956676852941
2020-08-03: 7657.502517198883
2020-08-04: 7512.508780152522
2020-08-05: 6610.499965846539
2020-08-06: 4317.573103638014
2020-08-07: 6694.336610893436
2020-08-08: 7733.240232726625
2020-08-09: 7938.086442812927
2020-08-10: 7809.8169776213335
2020-08-11: 7108.706307160727
2020-08-12: 6516.8982908974185
2020-08-13: 7610.628721040513
2020-08-14: 8288.02818089535
2020-08-15: 8487.57727730212
2020-08-16: 8402.621681997454
2020-08-17: 8067.693564021746
2020-08-18: 8025.158358299946
2020-08-19: 8630.741115633507
2020-08-20: 9086.95646979446
2020-08-21: 9274.562257816478
2020-08-22: 9272.588766722352
2020-08-23: 9212.189438221667
2020-08-24: 9349.989066072361
2020-08-25: 9755.859121185646
2020-08-26: 10097.425021365898
2020-08-27: 1

...

2020-08-11: 2716.8041801144586
2020-08-12: 2737.0592404979684
2020-08-13: 3330.480262658197
2020-08-14: 3584.096546117677
2020-08-15: 3532.368307950752
2020-08-16: 3376.236006115408
2020-08-17: 3276.2201504581194
2020-08-18: 3398.7915071096354
2020-08-19: 3724.901294722051
2020-08-20: 3893.77442168047
2020-08-21: 3897.2493447773772
2020-08-22: 3846.278605992495
2020-08-23: 3857.4768146202664
2020-08-24: 3992.2586803943304
2020-08-25: 4201.805658421723
2020-08-26: 4331.8865579594285
2020-08-27: 4370.139959430704
2020-08-28: 4382.767590535283
2020-08-29: 4443.2902400624325
2020-08-30: 4573.456217722328
2020-08-31: 4731.276789514471

Predicting for United States__Louisiana
2020-08-01: 2274.82689309633
2020-08-02: 2954.1143496139857
2020-08-03: 1691.252675475434
2020-08-04: 2506.68508017522
2020-08-05: 1504.4870448483534
2020-08-06: 1326.6538252082296
2020-08-07: 2359.738193887125
2020-08-08: 2603.1097566316034
2020-08-09: 2265.575041355621
2020-08-10: 2398.5261711450244
2020-08-11: 2012.2

2020-08-23: 1680.9441692175833
2020-08-24: 1738.795579640072
2020-08-25: 1830.1067242601298
2020-08-26: 1894.0316409454463
2020-08-27: 1924.847616918897
2020-08-28: 1942.1150835847475
2020-08-29: 1971.4107717086604
2020-08-30: 2030.4594501861452
2020-08-31: 2103.1589125815676

Predicting for United States__North Carolina
2020-08-01: 3383.950862515594
2020-08-02: 3658.565087244215
2020-08-03: 3727.5591709182713
2020-08-04: 3876.7758356079385
2020-08-05: 2691.0955208766186
2020-08-06: 2105.3417131225506
2020-08-07: 3463.2686034853136
2020-08-08: 3822.4773449425606
2020-08-09: 3931.3278297225474
2020-08-10: 3843.491004752325
2020-08-11: 3285.3836221221113
2020-08-12: 3212.2977935780186
2020-08-13: 3840.524987480312
2020-08-14: 4125.655259116956
2020-08-15: 4217.485689482688
2020-08-16: 4133.727795335238
2020-08-17: 3914.17119870196
2020-08-18: 3990.017211606039
2020-08-19: 4334.29550954343
2020-08-20: 4546.131831031637
2020-08-21: 4629.332004057025
2020-08-22: 4604.136817173224
2020-08-23

2020-08-03: 3512.51700112543
2020-08-04: 3504.9548054199013
2020-08-05: 3007.107937628843
2020-08-06: 1951.8504659617147
2020-08-07: 2963.8608408716477
2020-08-08: 3446.2157106729533
2020-08-09: 3648.050803916119
2020-08-10: 3617.547503516923
2020-08-11: 3266.423852768978
2020-08-12: 2978.6105550161296
2020-08-13: 3450.084997216842
2020-08-14: 3778.264910684809
2020-08-15: 3923.082738752321
2020-08-16: 3900.8375558538282
2020-08-17: 3736.889122050851
2020-08-18: 3701.443368555543
2020-08-19: 3969.684268583883
2020-08-20: 4195.3251570817765
2020-08-21: 4310.147654830093
2020-08-22: 4320.522063664702
2020-08-23: 4280.683498036534
2020-08-24: 4335.461287872225
2020-08-25: 4520.815189311342
2020-08-26: 4690.117656008223
2020-08-27: 4790.22992935572
2020-08-28: 4835.0198741306285
2020-08-29: 4867.981649349241
2020-08-30: 4959.377304735962
2020-08-31: 5110.192059143707

Predicting for United States__Oregon
2020-08-01: 1116.7775371506145
2020-08-02: 1383.6910016623397
2020-08-03: 1491.4656330

2020-08-02: 2487.059006854932
2020-08-03: 2430.432300074013
2020-08-04: 2528.6577126514926
2020-08-05: 2493.673338829085
2020-08-06: 1511.9124368304106
2020-08-07: 2243.3010478921274
2020-08-08: 2609.108705733744
2020-08-09: 2661.7617643585145
2020-08-10: 2721.1397804800113
2020-08-11: 2578.5175421151257
2020-08-12: 2299.8257895476186
2020-08-13: 2633.1236298875187
2020-08-14: 2864.6800360790317
2020-08-15: 2946.019870551379
2020-08-16: 2979.192133922683
2020-08-17: 2904.0185183319604
2020-08-18: 2856.7737869069274
2020-08-19: 3045.194559709539
2020-08-20: 3204.2128576176688
2020-08-21: 3287.4441614722814
2020-08-22: 3324.155445483789
2020-08-23: 3316.142931735247
2020-08-24: 3351.3419552756814
2020-08-25: 3484.0762401540137
2020-08-26: 3607.3442674039698
2020-08-27: 3686.0237522113885
2020-08-28: 3736.9573787630347
2020-08-29: 3773.739705849103
2020-08-30: 3842.168330753045
2020-08-31: 3953.7192400474864

Predicting for United States__Virgin Islands
2020-08-01: 57.724942568090476
2020

2020-08-14: 583.1256116522733
2020-08-15: 618.4128265924805
2020-08-16: 679.6894994464486
2020-08-17: 692.5174123329551
2020-08-18: 688.0422653671362
2020-08-19: 717.1579729642252
2020-08-20: 748.2963115713338
2020-08-21: 786.5268416254665
2020-08-22: 828.7724412329679
2020-08-23: 868.8825142027883
2020-08-24: 889.4933287646949
2020-08-25: 921.2381162592787
2020-08-26: 957.1683541031672
2020-08-27: 997.1988612644753
2020-08-28: 1039.3451463113513
2020-08-29: 1077.9964959944655
2020-08-30: 1110.2944043558705
2020-08-31: 1147.0686590794758

Predicting for United States Virgin Islands__nan
2020-08-01: 59.77380571350194
2020-08-02: 71.80606497862429
2020-08-03: 79.34636237680664
2020-08-04: 84.23796552884426
2020-08-05: 89.11291566951827
2020-08-06: 99.01411080811803
2020-08-07: 131.29505383258646
2020-08-08: 147.11936034139086
2020-08-09: 157.94058666578508
2020-08-10: 167.37696298531756
2020-08-11: 177.4955479372481
2020-08-12: 193.4453986419376
2020-08-13: 216.7059665502021
2020-08-14: 

In [37]:
# Check the predictions
preds_df.head()

,CountryName,RegionName,Date,PredictedDailyNewCases
213,Aruba,NaN,2020-08-01,58.823002
214,Aruba,NaN,2020-08-02,71.315828
215,Aruba,NaN,2020-08-03,78.993986
216,Aruba,NaN,2020-08-04,90.229279
217,Aruba,NaN,2020-08-05,87.810740


# Validation
This is how the predictor is going to be called during the competition.  
!!! PLEASE DO NOT CHANGE THE API !!!

In [38]:
!python predict.py -s 2020-08-01 -e 2020-08-04 -ip ../../../validation/data/2020-09-30_historical_ip.csv -o predictions/2020-08-01_2020-08-04.csv

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to predictions/2020-08-01_2020-08-04.csv
Done!


In [39]:
!head predictions/2020-08-01_2020-08-04.csv

CountryName,RegionName,Date,PredictedDailyNewCases
Aruba,,2020-08-01,58.82300181119936
Aruba,,2020-08-02,71.31582751123675
Aruba,,2020-08-03,78.99398601057783
Aruba,,2020-08-04,90.2292787574965
Afghanistan,,2020-08-01,149.81410233721954
Afghanistan,,2020-08-02,287.19834412144843
Afghanistan,,2020-08-03,277.60566758591983
Afghanistan,,2020-08-04,264.77630587676333
Angola,,2020-08-01,198.681737901836


# Test cases
We can generate a prediction file. Let's validate a few cases...

In [40]:
import os
from covid_xprize.validation.predictor_validation import validate_submission

def validate(start_date, end_date, ip_file, output_file):
    # First, delete any potential old file
    try:
        os.remove(output_file)
    except OSError:
        pass
    
    # Then generate the prediction, calling the official API
    !python predict.py -s {start_date} -e {end_date} -ip {ip_file} -o {output_file}
    
    # And validate it
    errors = validate_submission(start_date, end_date, ip_file, output_file)
    if errors:
        for error in errors:
            print(error)
    else:
        print("All good!")

## 4 days, no gap
- All countries and regions
- Official number of cases is known up to start_date
- Intervention Plans are the official ones

In [41]:
validate(start_date="2020-08-01",
         end_date="2020-08-04",
         ip_file="../../../validation/data/2020-09-30_historical_ip.csv",
         output_file="predictions/val_4_days.csv")

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to predictions/val_4_days.csv
Done!
All good!


## 1 month in the future
- 2 countries only
- there's a gap between date of last known number of cases and start_date
- For future dates, Intervention Plans contains scenarios for which predictions are requested to answer the question: what will happen if we apply these plans?

In [42]:
%%time
validate(start_date="2021-01-01",
         end_date="2021-01-31",
         ip_file="../../../validation/data/future_ip.csv",
         output_file="predictions/val_1_month_future.csv")

Generating predictions from 2021-01-01 to 2021-01-31...
Saved predictions to predictions/val_1_month_future.csv
Done!
All good!
CPU times: user 94.9 ms, sys: 44.4 ms, total: 139 ms
Wall time: 2.79 s


## 180 days, from a future date, all countries and regions
- Prediction start date is 1 week from now. (i.e. assuming submission date is 1 week from now)  
- Prediction end date is 6 months after start date.  
- Prediction is requested for all available countries and regions.  
- Intervention plan scenario: freeze last known intervention plans for each country and region.  

As the number of cases is not known yet between today and start date, but the model relies on them, the model has to predict them in order to use them.  
This test is the most demanding test. It should take less than 1 hour to generate the prediction file.

### Generate the scenario

In [43]:
from datetime import datetime, timedelta

start_date = datetime.now() + timedelta(days=7)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date = start_date + timedelta(days=180)
end_date_str = end_date.strftime('%Y-%m-%d')
print(f"Start date: {start_date_str}")
print(f"End date: {end_date_str}")

Start date: 2020-12-01
End date: 2021-05-30


In [44]:
from covid_xprize.validation.scenario_generator import get_raw_data, generate_scenario, NPI_COLUMNS
DATA_FILE = 'data/OxCGRT_latest.csv'
latest_df = get_raw_data(DATA_FILE, latest=True)
scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries=None, scenario="Freeze")
scenario_file = "predictions/180_days_future_scenario.csv"
scenario_df.to_csv(scenario_file, index=False)
print(f"Saved scenario to {scenario_file}")

Saved scenario to predictions/180_days_future_scenario.csv


### Check it

In [45]:
%%time
validate(start_date=start_date_str,
         end_date=end_date_str,
         ip_file=scenario_file,
         output_file="predictions/val_6_month_future.csv")

Generating predictions from 2020-12-01 to 2021-05-30...
Saved predictions to predictions/val_6_month_future.csv
Done!
All good!
CPU times: user 16.3 s, sys: 2.58 s, total: 18.8 s
Wall time: 5min 6s
